In [1]:
import sys

sys.path.append("../")

import pandas as pd
import numpy as np
import datetime
import os
from pprint import pprint
import matplotlib.pyplot as plt
import time
import vectorbtpro as vbt
from time import time
import helpers as pth
import platform
from dotenv import load_dotenv
import scipy.stats as stats
import time
import helpers as pth

theme = 'light'
vbt.settings.set_theme(theme)

pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 20)
# plt.rcParams["axes.grid"] = True
plt.rcParams["figure.figsize"] = (12, 7)
plt.rcParams["axes.formatter.useoffset"] = False
plt.rcParams["axes.formatter.limits"] = [-1000000000, 1000000000]
plt.style.use('classic' if theme == 'light' else 'dark_background')

if platform.system().lower() == "windows":
    base_data_path = "H:\\phitech-data\\01_raw"
else:
    from core_chains.simple.llm import make_Q_chain

    base_data_path = "../../phitech-data/01_raw"
    load_dotenv("../../sandatasci-core/credentials")
    Q = make_Q_chain("gpt-4o-instance1", __vsc_ipynb_file__)

In [2]:
%%html
<style>
.dataframe {
    font-size: 9pt; /* Adjust font size as needed */
}
</style>

In [3]:
symbols = ["MES", "6B"]
df = pth.SierraChartData.pull(
    symbols,
    timeframe="1min",
    start="2024-08-01",
    end="2024-12-01",
)
df

c:\Users\spask\miniconda3\Lib\site-packages\vectorbtpro\data\base.py:1728: VBTWarning: Symbols have mismatching index. Setting missing data points to NaN.


In [4]:
df.data['MES'].info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 119631 entries, 2024-08-01 00:00:00+00:00 to 2024-12-01 23:59:00+00:00
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   open         119331 non-null  float64
 1   high         119331 non-null  float64
 2   low          119331 non-null  float64
 3   close        119331 non-null  float64
 4   volume       119331 non-null  float64
 5   #_of_trades  119331 non-null  float64
 6   ohlc_avg     119331 non-null  float64
 7   hlc_avg      119331 non-null  float64
 8   hl_avg       119331 non-null  float64
 9   bid_volume   119331 non-null  float64
 10  ask_volume   119331 non-null  float64
dtypes: float64(11)
memory usage: 11.0 MB


In [5]:
high, low, close = df.get('High').dropna(), df.get('Low').dropna(), df.get('Close').dropna()
close

symbol,MES,6B
timestamp,,
2024-08-01 00:00:00+00:00,5714.75,1.2858
2024-08-01 00:01:00+00:00,5715.25,1.2858
2024-08-01 00:02:00+00:00,5715.50,1.2858
2024-08-01 00:03:00+00:00,5714.50,1.2857
2024-08-01 00:05:00+00:00,5713.75,1.2857
...,...,...
2024-12-01 23:55:00+00:00,6115.00,1.2689
2024-12-01 23:56:00+00:00,6115.25,1.2689
2024-12-01 23:57:00+00:00,6115.75,1.2690


In [6]:
# TODO: continue here when you're coming back to custom indicators in vectorbt
#       make sure you understand how the custom is working and how it's compiled.